<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-04-15 13:44:30
-------------------
qualified stocks: 91
with latest results: 85
still star stocks: 47
-------------------
Initial Investment:  1.09 C
CY Investment:  1.36 C
Reserve:  4.00 K
Current:  1.14 C
-------------------
Today PnL: 1.63 L (1.45%)
Current PnL: -22.11 L (-16.26%)
CY Booked + Current PnL: -21.98 L (-16.16%)
-------------------
Total profit:  3.74 L
Total loss:  -25.85 L
-------------------
Total Booked + Current PnL: 5.00 L (4.59%)
Total Booked PnL: 27.11 L (24.87%)
Curr Year Booked PnL: 13.50 K (0.12%)
Prev Year Booked PnL: 26.98 L (24.74%)
Est FTT:  1.94 C
Est FTT PnL: 79.60 L (69.74%)
-------------------
Est LTT:  2.55 C
Est LTT PnL: 1.41 C (123.31%)
Deployed:  1.09 C
Current:  1.14 C
CAGR/XIRR %: 3.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,AXISBANK,1003.18,1317.30,-19.21,H-LC,47.25,135542.0,13154.0,25170.0,3.93,10.75,18.57,31.31,NaN,0.52,1.19,17.18,X40,ATH,BANKS
71,SBIN,760.30,863.00,-22.31,M-LC,39.56,202804.0,1324.0,25898.0,1.52,0.66,12.77,13.51,93.0,0.05,1.78,11.11,XY25,NTT,BANKS
55,NMDC,65.62,82.00,31.22,M-MC,26.37,98410.0,-1989.0,27053.0,2.24,-1.98,27.49,24.96,507.0,-0.07,0.86,10.92,XY25,NTT,MINING
70,SBILIFE,1496.49,1924.99,-11.23,H-LC,64.84,131804.0,6099.0,29893.0,3.10,4.85,22.68,28.63,NaN,0.20,1.16,17.42,AR,ATH,INSURANCE
36,HINDUNILVR,2430.78,2723.00,-17.33,X-LC,62.64,198425.0,-5761.0,30299.0,-0.17,-2.82,15.27,12.02,-3.0,-0.19,1.74,9.50,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,SONACOMS,578.05,1006.00,-38.64,M-MC,9.89,80395.0,-20764.0,95654.0,6.81,-20.53,118.98,74.03,-8.0,-0.22,0.70,12.99,AR,BTT,AUTO
39,INDUSINDBK,1354.50,1800.00,-43.53,H-MC,59.34,41898.0,-35308.0,60702.0,6.61,-45.73,144.88,32.89,-21.0,-0.58,0.37,15.38,XR,NTT,BANKS
49,LUXIND,1448.19,2442.00,2.25,M-SC,93.41,136676.0,-902.0,95318.0,6.35,-0.66,69.74,68.62,6.0,-0.01,1.20,27.23,X40N,NTT,TEXTILES
88,VAIBHAVGBL,350.21,521.00,-31.60,X-SC,50.55,96526.0,-55465.0,129586.0,5.32,-36.49,134.25,48.77,-24.0,-0.43,0.85,10.87,XR,NTT,JEWELLERY
21,CERA,9059.90,10420.45,-25.30,X-SC,73.63,56770.0,-33829.0,47437.0,5.26,-37.34,83.56,15.02,-57.0,-0.71,0.50,11.70,X40N,ATH,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,QUESS,711.66,986.0,-67.94,M-SC,1.10,65306.0,-90548.0,150628.0,-50.70,-58.10,230.65,38.55,-33.0,-0.60,0.57,0.00,XY24,NTT,MISC
2,AAVAS,1815.63,2328.0,-28.66,M-SC,36.26,214653.0,20381.0,34452.0,-1.09,10.49,16.05,28.22,6.0,0.59,1.88,31.53,XY24,NTT,FINANCE
16,BERGEPAINT,558.54,680.0,-24.06,H-MC,34.07,166377.0,-10680.0,49181.0,-1.07,-6.03,29.56,21.75,-14.0,-0.22,1.46,19.01,XY24,NTT,PAINTS
19,CAMPUS,294.86,393.0,-36.31,M-SC,15.38,134261.0,-33514.0,89351.0,-0.99,-19.98,66.55,33.28,-16.0,-0.38,1.18,3.95,XY24,NTT,FOOTWEAR
75,SIS,477.00,528.0,10.48,M-SC,79.12,56389.0,-24701.0,33371.0,-0.93,-30.46,59.18,10.69,-28.0,-0.74,0.49,14.28,X40N,NTT,MISC


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NHPC,82.08,115.00,209.52,M-MC,58.24,144656.0,3643.0,52915.0,-0.24,2.58,36.58,40.11,-54.0,0.07,1.27,18.26,XY24,NTT,POWER
29,GICRE,335.06,583.00,0.21,H-MC,31.87,178340.0,31919.0,76437.0,2.71,21.80,42.86,74.00,109.0,0.42,1.56,30.43,XY24,BTT,INSURANCE
20,CAMS,3643.00,5250.99,-4.19,H-SC,70.33,106610.0,4606.0,40416.0,2.44,4.52,37.91,44.14,261.0,0.11,0.93,27.48,XR,ATH,MISC
5,ADANIPORTS,1103.69,1583.00,-6.22,M-LC,42.86,165469.0,14263.0,51395.0,3.71,9.43,31.06,43.43,NaN,0.28,1.45,14.70,XY24,NTT,MISC
68,SBICARD,765.91,1075.00,-6.61,H-MC,29.67,205742.0,25753.0,46889.0,2.76,14.31,22.79,40.36,-1.0,0.55,1.80,32.28,XY24,NTT,FINANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,CAMS,3643.00,5250.99,-4.19,H-SC,70.33,106610.0,4606.0,40416.0,2.44,4.52,37.91,44.14,261.0,0.11,0.93,27.48,XR,ATH,MISC
14,BANKINDIA,112.90,190.00,-30.51,M-MC,90.11,97866.0,-2389.0,70855.0,1.89,-2.38,72.40,68.29,-12.0,-0.03,0.86,21.98,XR,NTT,BANKS
1,AARTIIND,487.04,919.00,6.30,M-SC,27.47,65882.0,-15454.0,87590.0,3.02,-19.00,132.95,88.69,-8.0,-0.18,0.58,11.10,XR,NTT,CHEMICALS
76,SONACOMS,578.05,1006.00,-38.64,M-MC,9.89,80395.0,-20764.0,95654.0,6.81,-20.53,118.98,74.03,-8.0,-0.22,0.70,12.99,AR,BTT,AUTO
66,SAMMAANCAP,170.35,326.00,-157.18,M-SC,46.15,67704.0,-34506.0,127893.0,4.68,-33.76,188.90,91.37,NaN,-0.27,0.59,9.91,XR,NTT,FINANCE


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HDFCLIFE,560.54,897.0,-4.15,H-LC,74.73,185481.0,38059.0,50432.0,2.87,25.82,27.19,60.02,16.0,0.75,1.63,32.41,X40,BTT,INSURANCE
51,MEDANTA,1022.60,1486.0,10.98,X-MC,53.85,193557.0,38122.0,32324.0,0.27,24.53,16.70,45.32,8.0,1.18,1.70,31.70,XY24,NTT,HEALTHCARE
68,SBICARD,765.91,1075.0,-6.61,H-MC,29.67,205742.0,25753.0,46889.0,2.76,14.31,22.79,40.36,-1.0,0.55,1.80,32.28,XY24,NTT,FINANCE
2,AAVAS,1815.63,2328.0,-28.66,M-SC,36.26,214653.0,20381.0,34452.0,-1.09,10.49,16.05,28.22,6.0,0.59,1.88,31.53,XY24,NTT,FINANCE
34,HDFCBANK,1541.80,2170.0,1.69,X-LC,63.74,239091.0,41741.0,38661.0,3.38,21.15,16.17,40.74,4.0,1.08,2.10,30.81,X40,BTT,BANKS


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.0,-271.15,H-SC,100.0,92911.0,-40375.0,124965.0,0.15,-30.29,134.50,63.47,-27.0,-0.32,0.81,55.74,XR,NTT,CHEMICALS
59,RAJESHEXPO,517.65,583.0,769.13,M-SC,98.9,51012.0,-86165.0,103483.0,2.59,-62.81,202.86,12.62,-49.0,-0.83,0.45,26.87,X40N,NTT,JEWELLERY
78,SURYODAY,189.78,240.0,-41.57,H-SC,97.8,94492.0,-55434.0,95106.0,3.07,-36.97,100.65,26.46,-41.0,-0.58,0.83,21.05,XR,NTT,BANKS
0,5PAISA,564.75,810.0,-50.88,H-SC,96.7,124787.0,-62145.0,143318.0,2.74,-33.24,114.85,43.43,20.0,-0.43,1.09,19.38,X40N,BTT,FINANCE
10,ATULAUTO,624.44,844.0,-37.12,M-SC,95.6,115947.0,-31421.0,83238.0,1.62,-21.32,71.79,35.16,-49.0,-0.38,1.02,18.43,XY24,NTT,AUTO


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,42.15
2,50,71.76


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    41.53
X40     13.45
X40N    12.70
AR      11.80
XY25     9.11
XR       9.06
SR       1.24
X200     1.13
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    20.49
M-SC    19.14
H-SC    18.66
M-MC     7.71
H-LC     7.53
M-LC     6.66
H-MC     6.46
X-MC     6.44
X-SC     2.78
L-SC     2.39
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,13.85,-11.32,63.33
FINANCE,11.08,-17.02,66.55
FMCG,10.76,-1.16,32.58
IT,6.73,-44.62,116.69
INSURANCE,6.68,2.19,43.14
MISC,6.40,-40.89,91.87
PAINTS,6.00,-20.42,56.06
CHEMICALS,4.19,-13.22,87.83
ELECTRICAL,3.83,-23.61,78.01


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3369477.0,31
X40N,1228920.0,16
XR,1203594.0,14
AR,1056111.0,11
X40,612441.0,10
SR,203830.0,2
XY25,196472.0,6
X200,88899.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2237245.0,18
M-SC,2161555.0,22
X-LC,891871.0,14
M-MC,663277.0,7
X-SC,360658.0,4
H-MC,338109.0,5
L-SC,325205.0,4
H-LC,297750.0,6
M-LC,296943.0,5


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1075599.0      9
H-SC       XY24       981554.0      6
           X40N       557259.0      6
M-MC       XY24       469715.0      4
M-SC       XR         468256.0      5
X-LC       X40        399713.0      6
H-SC       AR         397979.0      2
M-SC       X40N       365646.0      5
H-SC       XR         300453.0      4
H-MC       XY24       277407.0      4
M-LC       XY24       271045.0      4
X-LC       AR         216165.0      3
L-SC       XR         173742.0      2
X-MC       X40N       130426.0      2
X-SC       XR         129586.0      1
H-LC       AR         124618.0      2
X-SC       SR         120329.0      1
X-LC       XY25       110837.0      3
X-SC       X40N       110743.0      2
M-SC       AR          96803.0      1
M-MC       AR          95654.0      1
X-LC       X200        88899.0      1
M-SC       SR          83501.0      1
L-SC       XY24        83077.0      1
X-LC       XY24        76257.0      1
H-LC       X40         75602.0      2
M-SC       X40         71750.0      1
M-MC       XR          70855.0      1
L-SC       AR          68386.0      1
X-MC       X40         65376.0      1
H-LC       X40N        64846.0      1
H-MC       XR          60702.0      1
X-MC       AR          56506.0      1
H-LC       XY25        32684.0      1
X-MC       XY24        32324.0      1
M-MC       XY25        27053.0      1
M-LC       XY25        25898.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
